In [11]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [12]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    
def train( model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))    

In [13]:
use_cuda = True

torch.manual_seed(1)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor()                       
                   ])),
    batch_size=64, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),                       
                   ])),
    batch_size=1000, shuffle=True, **kwargs)


model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.5)

for epoch in range(1, 11):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

torch.save(model.state_dict(),"mnist_cnn.pt")

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.301512
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.197820
Train Epoch: 1 [1280/60000 (2%)]	Loss: 1.759414
Train Epoch: 1 [1920/60000 (3%)]	Loss: 1.658545
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.966596
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.708537
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.348855
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.305927
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.118591
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.210858
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.154081
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.134231
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.135823
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.249036
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.179729
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.117621
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.066140
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.361238
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.261114
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.117776
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.058508
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.102859
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.077309
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.032149
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.043402
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.005440
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.039694
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.073005
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.007934
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.050318
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.011692
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.016637
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.082920
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.040979
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.013403
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.004892
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.014111
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.020555
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.006876
Train Epoch: 2 [58240/60000 (97%)]	Loss: 0.139008


Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.019645
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.013528
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.007955
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.006459
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.047130
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.071113
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.068864
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.057858
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.015111
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.039294
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.003163
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.020095
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.017422
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.000850
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.011832
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.089557
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.004452
Train Epoch: 4 [41600/60000 (69%)]	Loss: 0.010882
Train Epoch: 4 [42240/60000 (70%)]	Loss: 0.001666
Train Epoch: 4 [42880/60000 (71%)]	Loss: 0.003152


Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.001461
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.033394
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.001610
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.022067
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.003020
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.004259
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.012420
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.018925
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.001742
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.000367
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.003865
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.002604
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.000237
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.004954
Train Epoch: 6 [24320/60000 (41%)]	Loss: 0.000631
Train Epoch: 6 [24960/60000 (42%)]	Loss: 0.000633
Train Epoch: 6 [25600/60000 (43%)]	Loss: 0.029237
Train Epoch: 6 [26240/60000 (44%)]	Loss: 0.042356
Train Epoch: 6 [26880/60000 (45%)]	Loss: 0.001683
Train Epoch: 6 [27520/60000 (46%)]	Loss: 0.031872



Test set: Average loss: 0.0252, Accuracy: 9927/10000 (99%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.000154
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.000867
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.000692
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.012748
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.055243
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.005203
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.007176
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.000672
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.000391
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.001042
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.002731
Train Epoch: 8 [7040/60000 (12%)]	Loss: 0.000419
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.000276
Train Epoch: 8 [8320/60000 (14%)]	Loss: 0.000041
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.000176
Train Epoch: 8 [9600/60000 (16%)]	Loss: 0.003703
Train Epoch: 8 [10240/60000 (17%)]	Loss: 0.000611
Train Epoch: 8 [10880/60000 (18%)]	Loss: 0.000791
Train Epoch: 8 [11520/60000 (19%)]	Loss: 0.031557
Train Epoch: 8 [12

Train Epoch: 9 [44160/60000 (74%)]	Loss: 0.000192
Train Epoch: 9 [44800/60000 (75%)]	Loss: 0.019974
Train Epoch: 9 [45440/60000 (76%)]	Loss: 0.009499
Train Epoch: 9 [46080/60000 (77%)]	Loss: 0.018024
Train Epoch: 9 [46720/60000 (78%)]	Loss: 0.009515
Train Epoch: 9 [47360/60000 (79%)]	Loss: 0.000654
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.004285
Train Epoch: 9 [48640/60000 (81%)]	Loss: 0.000038
Train Epoch: 9 [49280/60000 (82%)]	Loss: 0.001793
Train Epoch: 9 [49920/60000 (83%)]	Loss: 0.000991
Train Epoch: 9 [50560/60000 (84%)]	Loss: 0.008195
Train Epoch: 9 [51200/60000 (85%)]	Loss: 0.000741
Train Epoch: 9 [51840/60000 (86%)]	Loss: 0.002351
Train Epoch: 9 [52480/60000 (87%)]	Loss: 0.000036
Train Epoch: 9 [53120/60000 (88%)]	Loss: 0.003132
Train Epoch: 9 [53760/60000 (90%)]	Loss: 0.003184
Train Epoch: 9 [54400/60000 (91%)]	Loss: 0.000408
Train Epoch: 9 [55040/60000 (92%)]	Loss: 0.004982
Train Epoch: 9 [55680/60000 (93%)]	Loss: 0.002371
Train Epoch: 9 [56320/60000 (94%)]	Loss: 0.000655


In [14]:
model = Net()
model

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

In [15]:
# CPU 용으로 해줘야 저장됨
state_dict = torch.load("mnist_cnn.pt")
model.load_state_dict(state_dict)
dummy_input  = torch.rand(1, 1, 28, 28)
torch.onnx.export(model, dummy_input, "torch_mnist.onnx")